In [ ]:
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import org.apache.spark.storage.StorageLevel


import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import org.apache.spark.storage.StorageLevel


In [ ]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [ ]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [ ]:
val connString = "DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net"
val account = "pipelines1"
val key = ""
def az(path: String): String = azurize("storage", account, path)

connString: String = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net
account: String = pipelines1
key: String = 
az: (path: String)String


In [ ]:
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@40299261


In [ ]:
import org.apache.spark.sql.functions._
import spark.implicits._

val toDouble = udf[Double, String]( _.toDouble)

import org.apache.spark.sql.functions._
import spark.implicits._
toDouble: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))


In [ ]:
val base = "/batch/quant"

base: String = /batch/quant


In [ ]:
import net.sansa_stack.rdf.spark.io._
import org.apache.jena.riot.Lang

val input = az("/go/go.owl")

val lang = Lang.RDFXML
val triples = spark.rdf(lang)(input)

http://purl.obolibrary.org/obo/go.owl @rdf:type owl:Ontology
http://purl.obolibrary.org/obo/go.owl @owl:versionIRI http://purl.obolibrary.org/obo/go/releases/2018-04-30/go.owl
http://purl.obolibrary.org/obo/go.owl @dc:license http://creativecommons.org/licenses/by/4.0/
http://purl.obolibrary.org/obo/go.owl @http://www.geneontology.org/formats/oboInOwl#default-namespace "gene_ontology"
http://purl.obolibrary.org/obo/go.owl @http://www.geneontology.org/formats/oboInOwl#hasOBOFormatVersion "1.2"
import net.sansa_stack.rdf.spark.io._
import org.apache.jena.riot.Lang
input: String = wasbs://storage@pipelines1.blob.core.windows.net//go/go.owl
lang: org.apache.jena.riot.Lang = Lang:RDF/XML
triples: org.apache.spark.rdd.RDD[org.apache.jena.graph.Triple] = MapPartitionsRDD[3] at map at package.scala:204


In [ ]:
import net.sansa_stack.rdf.spark.model._
import net.sansa_stack.query.spark.query

import org.apache.jena.graph.NodeFactory
def toGO(id: String) = NodeFactory.createURI(id.replace("GO:","http://purl.obolibrary.org/obo/GO_"))

import net.sansa_stack.rdf.spark.model._
import net.sansa_stack.query.spark.query
import org.apache.jena.graph.NodeFactory
toGO: (id: String)org.apache.jena.graph.Node
f: org.apache.spark.rdd.RDD[org.apache.jena.graph.Triple] = MapPartitionsRDD[55] at filter at TripleOps.scala:127


In [ ]:
import net.sansa_stack.rdf.spark.model.graph._
val g = GraphOps.constructStringGraph(triples).cache
g

import net.sansa_stack.rdf.spark.model.graph._
g: org.apache.spark.graphx.Graph[String,String] = org.apache.spark.graphx.impl.GraphImpl@6d6ed8e6
res73: org.apache.spark.graphx.Graph[String,String] = org.apache.spark.graphx.impl.GraphImpl@6d6ed8e6


org.apache.spark.graphx.impl.GraphImpl@6d6ed8e6

In [ ]:
import org.apache.jena.graph.NodeFactory
def uri(str: String) = NodeFactory.createURI(str)
def toGO(id: String) = uri(id.replace("GO:","http://purl.obolibrary.org/obo/go#"))
val alternative = uri("http://www.geneontology.org/formats/oboInOwl#hasAlternativeId")
val namespace =  uri("http://www.geneontology.org/formats/oboInOwl#hasOBONamespace")


import org.apache.jena.graph.NodeFactory
uri: (str: String)org.apache.jena.graph.Node
toGO: (id: String)org.apache.jena.graph.Node
alternative: org.apache.jena.graph.Node = http://www.geneontology.org/formats/oboInOwl#hasAlternativeId
namespace: org.apache.jena.graph.Node = http://www.geneontology.org/formats/oboInOwl#hasOBONamespace


In [ ]:
val namespaces = triples.find(None, Some(namespace), None).cache()
val alternatives = triples.find(None, Some(alternative), None).cache()

namespaces: org.apache.spark.rdd.RDD[org.apache.jena.graph.Triple] = MapPartitionsRDD[58] at filter at TripleOps.scala:127
alternatives: org.apache.spark.rdd.RDD[org.apache.jena.graph.Triple] = MapPartitionsRDD[59] at filter at TripleOps.scala:127


In [ ]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
  .option("sep", sep)
  .option("header", header)
  .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [ ]:
val comparison = readTSV(az("expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts.tsv"), true)
comparison

comparison: org.apache.spark.sql.DataFrame = [go: string, label: string ... 26 more fields]
res60: org.apache.spark.sql.DataFrame = [go: string, label: string ... 26 more fields]


In [ ]:
import net.sansa_stack.rdf.spark.utils.NodeUtils
val trips = triples.map{
  t=>(NodeUtils.getNodeValue(t.getSubject) , NodeUtils.getNodeValue(t.getPredicate), NodeUtils.getNodeValue(t.getObject))
}.cache()
trips.count

import net.sansa_stack.rdf.spark.utils.NodeUtils
trips: org.apache.spark.rdd.RDD[(String, String, String)] = MapPartitionsRDD[111] at map at <console>:123
res89: Long = 1582297


1582297

In [ ]:
val goTerms = trips.filter(_._1.contains("GO_")).map{ case (s, p ,b) => s -> (p, b)}.cache()
goTerms.count

goTerms: org.apache.spark.rdd.RDD[(String, (String, String))] = MapPartitionsRDD[115] at map at <console>:134
res95: Long = 586480


586480

In [ ]:
val alts = goTerms
  .filter{ case (_, (p, _)) => p == "http://www.geneontology.org/formats/oboInOwl#hasAlternativeId"}
    .mapValues{ case (p, o) => o}.reduceByKey{ case (a, b) => a + "; " + b}.cache()

val nms = goTerms
  .filter{ case (_, (p, _)) => p == "http://www.geneontology.org/formats/oboInOwl#hasOBONamespace"}
  .mapValues{ case (p, o) => o}.reduceByKey{ case (a, b) => a + "; " + b}.cache()

(alts.count, nms.count)

alts: org.apache.spark.rdd.RDD[(String, String)] = ShuffledRDD[118] at reduceByKey at <console>:136
nms: org.apache.spark.rdd.RDD[(String, String)] = ShuffledRDD[121] at reduceByKey at <console>:140
res97: (Long, Long) = (1706,47167)


(1706,47167)

In [ ]:
comparison

res173: org.apache.spark.sql.DataFrame = [go: string, label: string ... 26 more fields]


In [ ]:
def toGO(str: String) = str.replace("http://purl.obolibrary.org/obo/GO_", "GO:")
val ns = nms.map{ case (key, value) => toGO(key) -> value}.toDF("go", "namespace")
val newColumns = "namespace"::comparison.columns.toList.tail.map(s=> if(s=="gray_whale_average") "gray_whale_avg" else s)
val table = comparison.join(ns, Seq("go"), "left_outer")
  .withColumn("gray_whale_avg", toDouble($"gray_whale_average")).drop("gray_whale_average")
  .select("go", newColumns:_*)
  .sort($"gray_whale_avg".desc)
table

toGO: (str: String)String
ns: org.apache.spark.sql.DataFrame = [go: string, namespace: string]
newColumns: List[String] = List(namespace, label, gray_whale_avg, uniref90_count, taxons_count, transcripts_count, human_transcripts_count, mouse_transcripts_count, cow_transcripts_count, type, gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)
table: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [go: string, namespace: string ... 27 more fields]
res186: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [go: string, na...

In [ ]:
(table.count, comparison.count)

res188: (Long, Long) = (18597,18597)


(18597,18597)

In [ ]:
val where = "file:///pipelines/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"

where: String = file:///pipelines/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv


In [ ]:
def writeTSV(dataFrame: DataFrame, path: String, header: Boolean = true, sep: String = "\t"): Unit =
  dataFrame.write.option("sep", sep).option("header",header).csv(path)


writeTSV: (dataFrame: org.apache.spark.sql.DataFrame, path: String, header: Boolean, sep: String)Unit


In [ ]:
writeTSV(table.coalesce(1), where)